this notebook is used to visiualize and analize shock regions

In [ ]:
from postprocessing import *
import glob
import os
import healpy as hp
from natsort import natsorted

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
density = '20'
mach    = '4'
jet     = '40'
start   = '15'

In [ ]:
simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
filename = "snap_%03d.hdf5" % (1)
shock_data = h5py.File(output_directory + filename, "r")
shock = shock_data['PartType0/Machnumber'][:]
energy = shock_data['PartType0/EnergyDissipation'][:]
jet_tracer = shock_data['PartType0/Jet_Tracer'][:]

In [ ]:
mask_jet = jet_tracer > 1e-5
mask_energy = energy > 0
# bins = np.logspace(1, 2, 51)
bins = np.logspace(-3, 9, 51)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
plt.hist(energy, bins=bins)
plt.xscale('log')
plt.yscale('log')

In [ ]:
i_file = 0
while True:
    i_file += 1
    
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        shock_data = h5py.File(output_directory + filename, "r")
    except:
        break

    shock = shock_data['PartType0/Machnumber'][:]
    bins = np.linspace(0, 150, 35)

    fig, ax = plt.subplots(2, 3, figsize=(11, 6.1))
    fig.tight_layout(w_pad=7.0, h_pad=3.0)
    plot_dens_vel(ax[0][0], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5)
    plot_temp_vel(ax[0][1], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5)
    plot_pressure(ax[0][2], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5)
    plot_jet_tracer(ax[1][0], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5)
    plot_shocks(ax[1][1], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5)
    #plot_shocks(ax[1][2], output_directory + "snap_%03d.hdf5" % (i_file), value='energy', fac=0.5)
    plot_radvelocity(ax[1][2], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5)

#     ax[1][2].hist(shock[shock != 0], bins=bins, log=True)
#     ax[1][2].set_xlabel('shock value distribution')
#     ax[1][2].set_xlim(0, 150)
#     ax[1][2].set_ylim(1, 1500)
    plt.savefig(figures_directory + f'6panels_{i_file}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/6panels*.png'
ofilename = figures_directory + '/6panels-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)

## mollview

In [ ]:
def find_shocks(directory, i_file):
    filename = "shocks_%03d.hdf5" % (i_file)
    snap_data = h5py.File(directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    shocks = snap_data['PartType0/Machnumber'][:].T
    return shocks, x - 500, y - 500, z - 500, get_time_from_snap(snap_data) * unit_time_in_megayr

In [ ]:
i_file = 1
shock_all, x_all, y_all, z_all, time = find_shocks(output_directory, i_file)

In [ ]:
radius_range = np.linspace(40, 500, 31)
NSIDE = 31
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
vec = hp.ang2vec(theta,phi)  # return unit 3D position vector

In [ ]:
for radius in radius_range:
    print(radius)
    vec_scaled = vec * radius
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + 20) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - 20))
    shock, x, y, z = shock_all[mask], x_all[mask], y_all[mask], z_all[mask]
    shock_shell = []

    for vector in vec_scaled:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        shock_shell.append(shock[distance.argsort()[:3]].mean())
    


    hp.mollview(np.array(shock_shell), title=fr"Shocks in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
            min=1e-2, max=100, rot=(90,0,0), unit="mass flux in cgs", norm='log', cmap='magma')
    hp.graticule()
    plt.savefig(figures_directory + f'shocks_mollview_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + f'/shocks_mollview_{i_file}_*.png'
ofilename = figures_directory + f'/shocks_mollview_{i_file}-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)

## shock area

In [ ]:
density = '100'
mach    = '4'
jet     = '44'
start   = '12'

In [ ]:
simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/jet{jet}_{start}')
#simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/turb_alter_{start}')

output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
filename = "snap_%03d.hdf5" % (3)
shock_data = h5py.File(output_directory + filename, "r")
shock = shock_data['PartType0/Machnumber'][:]
masses = shock_data['PartType0/Masses'][:]
energy = shock_data['PartType0/EnergyDissipation'][:]
density_grad = shock_data['PartType0/DensityGradient'][:]
jet_tracer = shock_data['PartType0/Jet_Tracer'][:]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11, 5.1))
fig.tight_layout(w_pad=7.0, h_pad=3.0)
plot_jet_tracer(ax[0], output_directory + "snap_%03d.hdf5" % (7), fac=0.5)
plot_jet_tracer(ax[1], output_directory + "snap_%03d.hdf5" % (7), edge=True, fac=0.5)

In [ ]:
mask_edge = (jet_tracer > 1e-5)# & (jet_tracer < 1e-2)

In [ ]:
bins = np.logspace(-5, 11, 51)

fig, ax = plt.subplots(figsize=(7, 5))
plt.hist(energy, weights=masses, bins=bins, label='whole box')
plt.hist(energy[mask_edge], weights=masses[mask_edge], bins=bins, label='edges of jet')
plt.xlabel('Energy Dissipation')
plt.ylabel(r'Mass [M$_\odot$]')
plt.ylim(10, 1e7)
plt.xscale('log')
plt.yscale('log')
plt.legend()

In [ ]:
bins = np.logspace(0, 2.5, 51)

fig, ax = plt.subplots(figsize=(7, 5))
plt.hist(shock, weights=masses, bins=bins, label='whole box')
plt.hist(shock[mask_edge], weights=masses[mask_edge], bins=bins, label='edges of jet')
plt.xlabel('Mach Number')
plt.ylabel(r'Mass [M$_\odot$]')
plt.ylim(10, 5e7)
plt.xscale('log')
plt.yscale('log')
plt.legend()

In [ ]:
density_grad = np.sqrt(density_grad[:,0] ** 2 + density_grad[:, 1] ** 2 + density_grad[:, 2] ** 2)

In [ ]:
bins = np.logspace(-8, 2, 51)

fig, ax = plt.subplots(figsize=(7, 5))
plt.hist(density_grad, weights=masses, bins=bins, label='whole box')
plt.hist(density_grad[mask_edge], weights=masses[mask_edge], bins=bins, label='edges of jet')
plt.xlabel('Density Gradient')
plt.ylabel(r'Mass [M$_\odot$]')
plt.ylim(10, 5e9)
plt.xscale('log')
plt.yscale('log')
plt.legend()